# Module Import

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

# Data Pull using Kaggle API

In [4]:
# Needs api files .kaggle location
import kaggle

In [5]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [6]:
api=KaggleApi()
api.authenticate()

In [7]:
api.dataset_download_file('mkechinov/ecommerce-behavior-data-from-multi-category-store',file_name='2019-Nov.csv')
# True output means api is called and data is saved in a zipped format in the same location

False

In [8]:
import zipfile

In [9]:
with zipfile.ZipFile('2019-Nov.csv.zip','r') as zipref:
    zipref.extractall()

In [2]:
import pandas as pd
data = pd.read_csv('2019-Nov.csv')

In [11]:
data.head(2)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
1,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283


In [12]:
data.apply(lambda x: x.unique())

event_time       [2019-11-01 00:00:00 UTC, 2019-11-01 00:00:01 ...
event_type                                  [view, cart, purchase]
product_id       [1003461, 5000088, 17302664, 3601530, 1004775,...
category_id      [2053013555631882655, 2053013566100866035, 205...
category_code    [electronics.smartphone, appliances.sewing_mac...
brand            [xiaomi, janome, creed, lg, hp, rondell, miche...
price            [489.07, 293.65, 28.31, 712.87, 183.27, 360.09...
user_id          [520088904, 530496790, 561587266, 518085591, 5...
user_session     [4d3b30da-a5e4-49df-b1a8-ba5943f1dd33, 8e5f4f8...
dtype: object

In [13]:
data.nunique()

event_time        2549559
event_type              3
product_id         190662
category_id           684
category_code         129
brand                4200
price               60435
user_id           3696117
user_session     13776050
dtype: int64

In [74]:
# data.groupby('brand').count()

In [3]:
xiami=data[data['brand']=='aardwolf']

In [4]:
xiami.to_csv('data.csv')

In [75]:
len(xiami)

478

In [76]:
xiami.apply(lambda x: x.unique())

event_time       [2019-11-02 02:38:36 UTC, 2019-11-02 14:57:53 ...
event_type                                  [view, cart, purchase]
product_id       [30901103, 30901021, 30900236, 30900483, 30900...
category_id                                  [2053013554004492609]
category_code                        [computers.components.cooler]
brand                                                   [aardwolf]
price            [79.15, 8.98, 25.35, 13.8, 35.14, 23.46, 23.3,...
user_id          [560959712, 524336628, 519283040, 519208780, 5...
user_session     [9d5757b3-f041-4e2d-aa06-2774750a8eba, 97bafe6...
dtype: object

In [77]:
xiami.nunique()

event_time       477
event_type         3
product_id         9
category_id        1
category_code      1
brand              1
price             21
user_id          175
user_session     272
dtype: int64

# Data Cleaning & Manipulation checks in python 

In [78]:
xiami.dropna(axis=0,inplace=True)

In [79]:
len(xiami)

478

In [80]:
xiami.isna().sum()

event_time       0
event_type       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64

In [81]:
xiami.dtypes

event_time        object
event_type        object
product_id         int64
category_id        int64
category_code     object
brand             object
price            float64
user_id            int64
user_session      object
dtype: object

Drop:
1) brand


Manipulations:
1) Category code map to unknown
2) extract date from event_time
3) event_type-> proper format
4) extract part of category code


- Select only the relevant columns
- Take distinct rows from that
- aggregate to event_date, category_code level views, add to carts, purchasers, purchases, price

In [82]:
xiami=xiami[['event_time','event_type','category_code','product_id','price','user_id']].drop_duplicates()

In [83]:
def str_dot_index(text):
    return(text[text.find('.')+1:])

def first_half(text):
    return(text[:text.find('.')])

In [84]:
xiami['category']=xiami['category_code'].apply(first_half)
xiami['subcategory']=xiami['category_code'].apply(str_dot_index)

In [85]:
xiami['event_date']=pd.to_datetime(xiami['event_time'],format='%Y-%m-%d %H:%M:%S UTC')

In [86]:
xiami['event_date']=xiami['event_date'].apply(lambda x:x.date())

In [87]:
xiami.drop(['category_code','event_time'],axis=1,inplace=True)

In [88]:
len(xiami)

478

In [89]:
len(xiami.drop_duplicates())

286

In [91]:
xiami.apply(lambda x: x.unique())

event_type                                [view, cart, purchase]
product_id     [30901103, 30901021, 30900236, 30900483, 30900...
price          [79.15, 8.98, 25.35, 13.8, 35.14, 23.46, 23.3,...
user_id        [560959712, 524336628, 519283040, 519208780, 5...
category                                             [computers]
subcategory                                  [components.cooler]
event_date     [2019-11-02, 2019-11-03, 2019-11-06, 2019-11-0...
dtype: object

In [93]:
data=xiami.drop('price',axis=1).drop_duplicates()

In [94]:
product_price=xiami[['product_id','price']].drop_duplicates()

In [95]:
data.to_csv('data.csv',index=False)

In [96]:
product_price.to_csv('price.csv',index=False)

# Code validation to convert python ETL commands to PySpark

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, to_date
from pyspark.sql.types import StringType, DateType

In [19]:
spark = SparkSession.builder.appName('KaggleDataETL').getOrCreate()

# Assuming the data file is in the same location as the script
data = spark.read.csv(r"C:\Users\Revathi P\Documents\Full time Prep\Company wise prep\Walmart\Manager\Experiment codes\Kaggle API\2019-Nov.csv", header=True, inferSchema=True)

In [7]:
# Filter rows where brand is 'aardwolf' and drop rows with null values
xiami = data.filter(data['brand'] == 'aardwolf').na.drop()

In [8]:
# Selecting specific columns and removing duplicates
xiami = xiami.select('event_time', 'event_type', 'category_code', 'product_id', 'price', 'user_id').dropDuplicates()

In [9]:
# Define UDFs for category and subcategory
def first_half(text):
    return text.split('.')[0] if text else None

def str_dot_index(text):
    return text.split('.')[1] if text and '.' in text else None

first_half_udf = udf(first_half, StringType())
str_dot_index_udf = udf(str_dot_index, StringType())

In [10]:
# Applying UDFs
xiami = xiami.withColumn('category', first_half_udf(col('category_code'))) \
             .withColumn('subcategory', str_dot_index_udf(col('category_code')))

In [11]:
# Convert event_time to date
xiami = xiami.withColumn('event_date', to_date(col('event_time'), 'yyyy-MM-dd HH:mm:ss UTC'))

In [12]:
# Drop unnecessary columns
xiami = xiami.drop('category_code', 'event_time')

In [13]:
# Prepare the final data DataFrame
data = xiami.drop('price').dropDuplicates()

In [14]:
# Prepare product_price DataFrame
product_price = xiami.select('product_id', 'price').dropDuplicates()

# Querying big query tables using BigQuery API

In [7]:
# !pip install google-cloud-bigquery

In [2]:
import os
from google.cloud import bigquery

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="applied-groove-405905-09c24ef39226.json"

In [4]:
client=bigquery.Client()

In [5]:
sql_query="""
SELECT * FROM `applied-groove-405905.Airflow_Test.product_price`
"""

In [6]:
query_job=client.query(sql_query)

In [7]:
# !pip install db-dtypes

In [10]:
product_price=query_job.to_dataframe()

In [11]:
type(product_price)

pandas.core.frame.DataFrame

In [12]:
product_price.head(2)

,product_id,price
0,30900483,13.80
1,30900483,13.77


# Emailing using SMTP API

In [9]:
import smtplib
server=smtplib.SMTP('smtp.gmail.com',587)
server.starttls()
server.login('b.revathycool@gmail.com','apnr frik hyqg soas')

(235, b'2.7.0 Accepted')

In [17]:
server.sendmail(
    'b.revathycool@gmail.com',
    'revathi.boopathi15081995@gmail.com',
    f'Product ID {product_price["product_id"][2]} is {product_price["price"][2]}'
)
print('Mail sent!')


Mail sent!
